In [1]:
# import libraries
import pandas as pd
import numpy as np

#### 1. Read the Yale Bright Star Catalog and create a Pandas data frame object

In [2]:
import pandas as pd
df = pd.read_csv('bsc5.csv', header=None)

In [3]:
# add column names
df.columns = ['HR',	'Name', 'DM', 'HD',	'SAO', 'FK5', 'IRflag', 'r_IRflag',	'Multiple',	'ADS', 'ADScomp', 'VarID', 'RAh1900', 'RAm1900', 'RAs1900',	'DE-1900', 'DEd1900', 'DEm1900', 'DEs1900', 'RAh', 'RAm', 'RAs', 'DE-', 'DEd', 'DEm', 'DEs', 'GLON', 'GLAT', 'Vmag', 'n_Vmag', 'u_Vmag', 'B-V',	'u_B-V', 'U-B',	'u_U-B', 'R-I', 'n_R-I', 'SpType', 'n_SpType', 'pmRA', 'pmDE', 'n_Parallax', 'Parallax', 'RadVel', 'n_RadVel', 'l_RotVel', 'RotVel', 'u_RotVel', 'Dmag', 'Sep', 'MultID', 'MultCnt', 'NoteFlag']

#### 2. Drop all objects that do not have a trigonometric parallax

In [4]:
# get the unique values
df['Parallax'].unique()

array(['     ', '0.014', '0.047', '0.05', '0.067', '0.032', '0.044',
       '0.002', '0.072', '0.009', '0.023', '0.066', '0.017', '-0.002',
       '0.034', '-0.015', '0.006', '0.024', '0.015', '0.003', '0.029',
       '0.013', '0.022', '0.053', '0.134', '0.005', '0.011', '0.027',
       '0.049', '0.004', '0.153', '0.035', '0.016', '-0.014', '0.033',
       '0.001', '0.018', '0.03', '0.008', '0.038', '0.06', '0.02',
       '0.007', '-0.003', '0.028', '0.076', '0.031', '0.099', '0.043',
       '0.01', '0.058', '0.061', '0.046', '0.012', '0.048', '0.021',
       '0.037', '0.176', '0.145', '0.019', '0.065', '0.025', '0.051',
       '0.039', '0.136', '0.036', '0.041', '0.026', '0.056', '0.069',
       '-0.012', '-0.016', '-0.001', '0', '0.062', '0.081', '0.152',
       '0.119', '-0.005', '0.275', '0.107', '-0.023', '0.057', '0.074',
       '-0.019', '0.09', '0.045', '0.064', '0.093', '0.089', '0.077',
       '0.147', '0.052', '-0.004', '0.121', '-0.007', '0.059', '0.084',
       '0.103', '0

In [5]:
# replace blank spaces with NaN
df['Parallax'] = df['Parallax'].replace('     ', np.nan)

In [6]:
# initial number of null values
df['Parallax'].isna().sum()

5821

In [7]:
# drop rows without a parallax value
df.dropna(subset = ['Parallax'], axis = 0, inplace = True)

# reset indices
df.reset_index(drop = True, inplace = True)

In [8]:
# confirm there are no null values
df['Parallax'].isna().sum()

0

#### 3. Create a column called Distance that has the distance of the object in parsec

In [9]:
# convert Parallax to numeric
df['Parallax'] = pd.to_numeric(df['Parallax'], errors = 'coerce')

# create column and convert parallax to distance
df['Distance'] = 1 / df['Parallax']

In [10]:
# confirm column was added
df['Distance']

0        71.428571
1        21.276596
2        20.000000
3        14.925373
4        31.250000
           ...    
3284    100.000000
3285    142.857143
3286     62.500000
3287     76.923077
3288     28.571429
Name: Distance, Length: 3289, dtype: float64

#### 5. Create three columns x, y, and z that will have the distances to the stars in parsecs in the equatorial frame of reference. Write a routine that will do a coordinate transformation from a spherical coordinate system to a Cartesian system. In this Cartesian system, the x-axis is pointing to the vernal equinox and the x-y plane is the plane of the celestial equator. The z-axis should be pointing to the north celestial pole. You can use the library Astropy for your work.

In [11]:
# convert right ascension (longitude) columns to numerical and handle missing values
df['RAh'] = pd.to_numeric(df['RAh'], errors = 'coerce')
df['RAm'] = pd.to_numeric(df['RAm'], errors = 'coerce')
df['RAs'] = pd.to_numeric(df['RAs'], errors = 'coerce')

# convert declination (latitude) columns to numerical
df['DEd'] = pd.to_numeric(df['DEd'], errors = 'coerce')
df['DEm'] = pd.to_numeric(df['DEm'], errors = 'coerce')
df['DEs'] = pd.to_numeric(df['DEs'], errors = 'coerce')

In [12]:
# create the ra column having right ascension in decimal degrees
ra_decimal_degrees = (df['RAh']) + (df['RAm'] / 60) + (df['RAs'] / 3600)
ra_decimal_degrees = ra_decimal_degrees * 15
df.insert(22, 'ra', ra_decimal_degrees)

# create the dec column having declination in decimal degrees
declination_decimal_degress = (df['DEd']) + (df['DEm'] / 60) + (df['DEs'] / 3600)
df.insert(26, 'dec', declination_decimal_degress)

In [13]:
from astropy.coordinates import SkyCoord
import astropy.units as u

equatorial = SkyCoord(ra = df['ra'] * u.degree, dec = df['dec'] * u.degree, distance = abs(df['Distance']) * u.pc)

# convert to Cartesian
x = equatorial.cartesian.x
y = equatorial.cartesian.y
z = equatorial.cartesian.z

# create columns and populate
df['x'] = x
df['y'] = y
df['z'] = z

#### 6. Create a column called Color in your data frame. The colors could be named colors like red or blue. Or, the colors could be in hexadecimal format. Here is the color scheme according to spectral types - O (deepest blue), B (medium blue), A (light blue), F (green), G (yellow), K (orange), M (red), and any other star not having a spectral type black.

In [14]:
# initialize new column to store star colors
df['Color'] = None

# assign colors
idx = 0
for spectral_type in df['SpType']:
    if spectral_type[2] == "O":
        color = "darkblue"
    elif spectral_type[2] == "B":
        color = "mediumblue"
    elif spectral_type[2] == "A":
        color = "lightblue"
    elif spectral_type[2] == "F":
        color = "green"
    elif spectral_type[2] == "G":
        color = "yellow"
    elif spectral_type[2] == "K":
        color = "orange"
    elif spectral_type[2] == "M":
        color = "red"
    else:
        color = "black"
        
    # add star color to df
    df.at[idx, 'Color'] = color  

    idx += 1

In [15]:
# check that the column was added
df['Color']

0           orange
1           yellow
2           yellow
3           orange
4       mediumblue
           ...    
3284        yellow
3285    mediumblue
3286        orange
3287        orange
3288        yellow
Name: Color, Length: 3289, dtype: object

#### 7. Use Plotly to obtain the 3-D distribution of the stars.

In [16]:
import plotly.express as px

fig = px.scatter_3d(data_frame = df, x = 'x', y = 'y', z = 'z', opacity = 0.6)

fig.update_layout(
    scene = dict(
        xaxis_title = 'X Axis',
        yaxis_title = 'Y Axis',
        zaxis_title = 'Z Axis',
    ),
    title = '3D Distribution of the Stars'
)

fig.show()

#### 8. The stars should be color-coded.

In [17]:
color_scale = ["orange", "yellow", "mediumblue", "green", "lightblue", "red", "black", "darkblue"]

fig = px.scatter_3d(data_frame = df, x = 'x', y = 'y', z = 'z', color = 'Color', opacity = 0.7,
                    hover_name = 'HR',
                    hover_data = ['Distance', 'RadVel'],
                    color_discrete_sequence = color_scale)

fig.update_layout(
    scene = dict(
        xaxis_title = 'X Axis',
        yaxis_title = 'Y Axis',
        zaxis_title = 'Z Axis'),
        title = '3D Distribution of the Stars'
)

fig.show()

#### 9. If you hover over a star, you should get the following information - HR Number, Distance in parsec, and Radial Velocity in km/s.  

In [18]:
color_scale = ["orange", "yellow", "mediumblue", "green", "lightblue", "red", "black", "darkblue"]

fig = px.scatter_3d(data_frame = df, x = 'x', y = 'y', z = 'z', color = 'Color', opacity = 0.7,
                    hover_name = 'HR',
                    hover_data = ['Distance', 'RadVel'],
                    custom_data = ['HR', 'Distance', 'RadVel'],
                    color_discrete_sequence = color_scale)

fig.update_traces(hovertemplate = 'HR Number: %{customdata[0]}<br>'
                  'Distance (parsec): %{customdata[1]:.2f}<br>'
                  'Radial Velocity (km/s): %{customdata[2]:.2f}')

fig.update_layout(
    scene = dict(
        xaxis_title = 'X Axis',
        yaxis_title = 'Y Axis',
        zaxis_title = 'Z Axis'),
        title = '3D Distribution of the Stars'
)

fig.show()